In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2024-05-02 15:51:13.217433: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv('../data/userData.csv')
df

,food_name,time_of_day,weekday
0,sweet tea,Night,Thursday
1,chick-fil-a sandwich,Night,Saturday
2,chick-fil-a sandwich,Night,Saturday
3,chick-fil-a 8 count nugget,Night,Friday
4,chocolate chip muffin,Night,Tuesday
...,...,...,...
359,optimum nutrition scoop of whey,Evening,Sunday
360,cups of milk,Evening,Sunday
361,bags of oatmeal,Night,Tuesday
362,optimum nutrition shake with 2 cups of milk,Evening,Tuesday


In [12]:
data_encoded = pd.get_dummies(df[['time_of_day']]) # time of day is Night, Morning, Afternoon, Evening
data_encoded = pd.concat([df[['food_name']], data_encoded], axis=1) # food_name is a string 


# Convert to numpy arrays
X = data_encoded.drop(columns=['food_name']).values
y = data_encoded['food_name'].values


# X, y

data_encoded


,food_name,time_of_day_Afternoon,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
0,sweet tea,False,False,False,True
1,chick-fil-a sandwich,False,False,False,True
2,chick-fil-a sandwich,False,False,False,True
3,chick-fil-a 8 count nugget,False,False,False,True
4,chocolate chip muffin,False,False,False,True
...,...,...,...,...,...
359,optimum nutrition scoop of whey,False,True,False,False
360,cups of milk,False,True,False,False
361,bags of oatmeal,False,False,False,True
362,optimum nutrition shake with 2 cups of milk,False,True,False,False


In [46]:
# Initialize k value
k = 5  # You can adjust the value of k

current_meal = np.array([0, 0, 0, 1])

# Convert data to TensorFlow tensors
X_tf = tf.constant(X, dtype=tf.float32)
y_tf = tf.constant(y)

distances = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(tf.expand_dims(X, axis=0), tf.expand_dims(X, axis=1))), axis=2))

# Compute pairwise distances
distances = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(tf.expand_dims(X_tf, axis=0), tf.expand_dims(X_tf, axis=1))), axis=2))

# Get the top k indices
top_k_indices = tf.nn.top_k(tf.negative(distances), k=k)[1]

# Gather the labels of the top k neighbors
top_k_labels = tf.gather(y_tf, top_k_indices)

top_k_labels

<tf.Tensor: shape=(364, 5), dtype=string, numpy=
array([[b'sweet tea', b'chick-fil-a sandwich', b'chick-fil-a sandwich',
        b'chick-fil-a 8 count nugget', b'chocolate chip muffin'],
       [b'sweet tea', b'chick-fil-a sandwich', b'chick-fil-a sandwich',
        b'chick-fil-a 8 count nugget', b'chocolate chip muffin'],
       [b'sweet tea', b'chick-fil-a sandwich', b'chick-fil-a sandwich',
        b'chick-fil-a 8 count nugget', b'chocolate chip muffin'],
       ...,
       [b'sweet tea', b'chick-fil-a sandwich', b'chick-fil-a sandwich',
        b'chick-fil-a 8 count nugget', b'chocolate chip muffin'],
       [b'free birds burrito with chicken, rice and queso', b'sweet tea',
        b'3 chocolate doughnuts', b'sweet tea', b'collachies'],
       [b'sweet tea', b'chick-fil-a sandwich', b'chick-fil-a sandwich',
        b'chick-fil-a 8 count nugget', b'chocolate chip muffin']],
      dtype=object)>

In [ ]:
clean = pd.read_csv('../data/userDataClean.csv')

compare = pd.concat([df['food_name'], clean['food_name']], axis=1)

# rename columns

compare.columns = ['original', 'clean']

# sort columns

# sort values in each column alphabetically
compare = compare.apply(lambda x: x.sort_values().reset_index(drop=True))

# Print the sorted DataFrame
compare
